# Main analysis on api collected data to follow

In [123]:
# import libraries

from eod import *
"""
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import json
from pathlib import Path
import numpy as np
import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv
%matplotlib inline
"""
import os
import requests
import pandas as pd
import json
from pathlib import Path
import numpy as np
import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv
#import data viz libraries
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
#import altair as alt
%matplotlib inline
# import interactive panels libraries
import panel as pn
from panel.interact import interact
from panel import widgets
# pn.extension('plotly')
pn.extension()


In [124]:
#pn.extension('plotly')

In [125]:
#import hvplot.pandas

In [126]:
# Read the API keys
load_dotenv()

True

In [127]:
##########################################################
# Create data structures used throughout
##########################################################
# Define positions (indexes, algorithmitic portfolios, assets to evaluate
# Define currencies to review performance against
# Define evaluation calculations to perform
# All are manually coded in for first MVP

asset1 = 'SP500'
asset2 = 'SP500BDT'
asset5 = 'GlobalArtIndex'
asset6 = 'XAUUSD'
asset7 = 'BTCUSD'

currency1 = 'USD'
currency2 = 'XAU'
currency3 = 'BTC'

In [128]:
####################################################################
# Main Market Dataset Collection
# Equities
# Precious Metals
# Crpyto currencies
####################################################################
to_eod_symbol_dict = {
                    'SP500BDT' : 'SP500BDT.INDX',
                    'SP500' : 'GSPC.INDX',
                    'BTCUSD' : 'BTC-USD.CC',
                    'XAUUSD': 'GC.COMM'}
#from_eod_symbol_dict = {}
#for key, value in to_eod_symbol_dict.items():
#    from_eod_symbol_dict[value] = key

#print(to_eod_symbol_dict[asset1])

start_date = '2010-01-01'
asset1_data_USD_df = get_data(to_eod_symbol_dict[asset1],start_date)
print(asset1)
asset1_data_USD_df['Asset'] = asset1
#asset1_data_USD_df['Asset'] = asset1_data_USD_df['Asset'].replace([to_eod_symbol_dict[asset1],'SP500'])
#asset1_data_USD_df['Asset'].replace([to_eod_symbol_dict[asset1],asset1],inplace=True)
asset1_data_USD_df.head()

SP500


,Asset,AdjClosePrice
Date,,
2010-01-04,SP500,1132.9900
2010-01-05,SP500,1136.5200
2010-01-06,SP500,1137.1400
2010-01-07,SP500,1141.6899
2010-01-08,SP500,1144.9800


In [129]:
asset2_data_USD_df = get_data(to_eod_symbol_dict[asset2],start_date)
print(asset2)
asset2_data_USD_df['Asset'] = asset2
asset2_data_USD_df.head()

SP500BDT


,Asset,AdjClosePrice
Date,,
2010-01-01,SP500BDT,288.90
2010-01-04,SP500BDT,289.37
2010-01-05,SP500BDT,291.11
2010-01-06,SP500BDT,290.71
2010-01-07,SP500BDT,290.72


In [130]:
asset6_data_USD_df = get_data(to_eod_symbol_dict[asset6],start_date)
print(asset6)
asset6_data_USD_df['Asset'] = asset6
asset6_data_USD_df.head()

XAUUSD


,Asset,AdjClosePrice
Date,,
2010-01-03,XAUUSD,1095.2
2010-01-04,XAUUSD,1118.3
2010-01-05,XAUUSD,1118.7
2010-01-06,XAUUSD,1136.5
2010-01-07,XAUUSD,1133.7


In [131]:
asset7_data_USD_df = get_data(to_eod_symbol_dict[asset7],start_date)
print(asset7)
asset7_data_USD_df['Asset'] = asset7
asset7_data_USD_df.head()

BTCUSD


,Asset,AdjClosePrice
Date,,
2010-07-17,BTCUSD,0.0495
2010-07-18,BTCUSD,0.1000
2010-07-19,BTCUSD,0.1000
2010-07-20,BTCUSD,0.1000
2010-07-21,BTCUSD,0.1000


In [132]:
## Fetch ART Data ##
art_file_path = Path("../data_imports/Artprice_Indexes_Quarterly_data_Base100_January1998.csv")
art_index = pd.read_csv(art_file_path, encoding='utf-8', parse_dates=True, infer_datetime_format=True)
art_index.rename(columns=({"Unnamed: 0" : "Date"}), inplace=True)
art_index.dropna()
art_index.set_index("Date", drop=True, inplace=True)
#art_index.tail()
# first select art index of choice and scale up price by 100 & add position name column
gblart_index_col_list = ['Asset','AdjClosePrice']
gblart_index_df = pd.DataFrame(columns=gblart_index_col_list)
gblart_index_df['AdjClosePrice'] = (art_index.loc[:,'Global Index (USD)'])*100
gblart_index_df['Asset'] = 'GlobalArtIndex'
asset5_data_USD_df = gblart_index_df
asset5_data_USD_df.head()


,Asset,AdjClosePrice
Date,,
1998-01-01,GlobalArtIndex,10000
1998-04-01,GlobalArtIndex,10700
1998-07-01,GlobalArtIndex,10600
1998-10-01,GlobalArtIndex,9600
1999-01-01,GlobalArtIndex,10400


In [133]:
####################################################################
# EVERYTHING ABOVE WILL BE REVISED ONCE DATASETS FUNCTIONS CREATED
# BELOW STARTS THE FINAL VERSION OF ANALYTICS AND VISUALIZATIONS
####################################################################

# Define calculation, currency and asset structure for 
# currency conversion and main analysis

# hard code for now to test

calc_list = ['STD_200D','MA_200D','DailyReturns','CumulReturns']

asset1_calc_USD_df = pd.DataFrame()
asset1_calc_BTC_df = pd.DataFrame()
asset1_calc_XAU_df = pd.DataFrame()

asset2_calc_USD_df = pd.DataFrame()
asset2_calc_BTC_df = pd.DataFrame()
asset2_calc_XAU_df = pd.DataFrame()

asset5_calc_USD_df = pd.DataFrame()
asset5_calc_BTC_df = pd.DataFrame()
asset5_calc_XAU_df = pd.DataFrame()

asset6_calc_USD_df = pd.DataFrame()
asset6_calc_BTC_df = pd.DataFrame()
asset6_calc_XAU_df = pd.DataFrame()

asset7_calc_USD_df = pd.DataFrame()
asset7_calc_BTC_df = pd.DataFrame()
asset7_calc_XAU_df = pd.DataFrame()

eval_asset_dict = {
                asset1 : {
                        'data' : asset1_data_USD_df,
                        'USD' : asset1_calc_USD_df,
                        'BTC' : asset1_calc_BTC_df,
                        'XAU' : asset1_calc_XAU_df
                        }}


In [134]:
####################################################################
# CURRENCY CONVERSION FUNCTION WILL BE HERE IN THIS CELL
####################################################################




In [135]:
####################################################################
# ASSET ANALYSIS FUNCTION 
####################################################################

def analyze_asset(calc_list,wip_calc_df):
    """
    Market price data analysis and asset return calcs
    as defined in calc_list input on provided wip_calc_df
    of format: date, asset, adjclose
    """
    for calc_type in calc_list:
        # Do analysis on price trends before return calcs 
        # to align return calcs with same window
        if calc_type == 'STD_200D':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].rolling(window=200).std()
        elif calc_type == 'MA_200D':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].rolling(window=200).mean()
            wip_calc_df.dropna(inplace=True)
        elif calc_type == 'DailyReturns':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].pct_change()
            wip_calc_df.dropna(inplace=True)
        elif calc_type == 'CumulReturns':
            wip_calc_df[calc_type] = (1+wip_calc_df['DailyReturns']).cumprod() - 1
        else:
            print(f"What is {calc_type} doing here?!")        
    return(wip_calc_df)


In [139]:
####################################################################
# MAIN ANALYSIS LOOP 
####################################################################
# hard code for now
wip_calc_df = asset1_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset1_calc_USD_df = wip_calc_df
asset1_calc_BTC_df = wip_calc_df
asset1_calc_XAU_df = wip_calc_df

wip_calc_df = asset2_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset2_calc_USD_df = wip_calc_df
asset2_calc_BTC_df = wip_calc_df
asset2_calc_XAU_df = wip_calc_df

wip_calc_df = asset5_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset5_calc_USD_df = wip_calc_df
asset5_calc_BTC_df = wip_calc_df
asset5_calc_XAU_df = wip_calc_df

wip_calc_df = asset6_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset6_calc_USD_df = wip_calc_df
asset6_calc_BTC_df = wip_calc_df
asset6_calc_XAU_df = wip_calc_df

wip_calc_df = asset7_data_USD_df
analyze_asset(calc_list,wip_calc_df)
asset7_calc_USD_df = wip_calc_df
asset7_calc_BTC_df = wip_calc_df
asset7_calc_XAU_df = wip_calc_df

"""
#loop for each asset: access proper df, currency convert, and analyze
for asset in eval_asset_list:
    print(f"Analyzing {asset}")
    for currency in currency_list: 
        # Run conversion on USD market price datasets to normalize price to alt currency
        if currency == 'USD':
            print(f"Market data in USD, not converting")
            wip_calc_df = asset1_data_USD_df
            analyze_asset(calc_list,wip_calc_df)
            asset1_calc_USD_df = wip_calc_df
        elif currency == 'XAU':
            print(f"Converting market price data to {currency}")
            wip_calc_df = asset1_data_USD_df
            analyze_asset(calc_list,wip_calc_df)
            asset1_calc_XAU_df = wip_calc_df
        elif currency == 'BTC':
            print(f"Converting market price data to {currency}")
            wip_calc_df = asset1_data_USD_df
            analyze_asset(calc_list,wip_calc_df)
            asset1_calc_BTC_df = wip_calc_df
        else:
            print(f"What is {currency} doing here?!")
"""
print(asset2_calc_USD_df.head())
print(asset2_calc_XAU_df.head())
print(asset2_calc_BTC_df.head())

               Asset  AdjClosePrice  STD_200D    MA_200D  DailyReturns  \
Date                                                                     
2013-01-25  SP500BDT         367.70  7.439383  361.96410     -0.004925   
2013-01-28  SP500BDT         367.42  7.390430  362.05705     -0.000761   
2013-01-29  SP500BDT         367.11  7.334046  362.15105     -0.000844   
2013-01-30  SP500BDT         366.63  7.272629  362.24360     -0.001308   
2013-01-31  SP500BDT         366.89  7.214010  362.33555      0.000709   

            CumulReturns  
Date                      
2013-01-25     -0.004925  
2013-01-28     -0.005683  
2013-01-29     -0.006522  
2013-01-30     -0.007821  
2013-01-31     -0.007117  
               Asset  AdjClosePrice  STD_200D    MA_200D  DailyReturns  \
Date                                                                     
2013-01-25  SP500BDT         367.70  7.439383  361.96410     -0.004925   
2013-01-28  SP500BDT         367.42  7.390430  362.05705     -0.000761

In [118]:
####################################################################
# CELL RESERVED FOR FINAL DF MASSAGE AS NEEDED FOR VISUALIZATIONS 
####################################################################

# After looping can concat into long frame as needed by visualizations
# lcombo_USD_df = pd.concat(frames)

In [119]:
####################################################################
# HENRY SCRATCH PAD SAVE FOR NEAR TERM REFERENCE
####################################################################
# test merge trying zero fill NaN locations to help with line plot below -- spoiler alert -- doesnt work
#test_combo_df = sp500_daily_returns.merge(gblart_returns_df,how='left',left_index=True,right_index=True).fillna(0)
# long head here to verify that its joining where there's valid art data
#test_combo_df.head(100)

#def sanitize_dataset(dataset_df):
#    """Insure data sorted ascending"""
#    dataset_df = dataset_df.sort_index()
#    print(dataset_df.head())
#    return(dataset_df)
#
## Sweep each position,for each currency, for each calculation
#for pos in eval_pos_list:
#    # first do finalize sanitation step on incoming position dataset
#    print(type(pos))
#    sanitize_dataset(pos)
#    for currency in currency_df_list:
#        if (currency == USD):


In [120]:
####################################################################
# THIS CELL IS START OF VISUALIZATION FUNCTIONS 
#
####################################################################
# Define Panel Visualization Functions
# Number of plot functions right now = 5 calcs/position * 2timeframes * 6 positions = 60!!!!

#def 
#art_index_std_bar = px.bar(art_index_std, title="Art Price STD 1998-2020")
#art_index_std_bar

In [121]:
# Create Panel Dashboard

